In [1]:
import subprocess
import re
import os
import sys
from typing import Optional, List


- `subprocess`: 터미널에서 명령어를 실행할 수 있게 해줍니다. 
    - 예를 들어, `git status` 같은 명령어를 Python으로 실행할 수 있습니다.
- `re`: 정규 표현식(특정 패턴 검사)을 처리합니다. 커밋 메시지가 규칙에 맞는지 확인할 때 사용됩니다.

- `os`: 운영 체제 관련 기능을 제공합니다.
    - 예를 들어, 파일이나 디렉토리가 존재하는지 확인할 때 유용합니다.
- `sys`: 시스템 관련 정보를 가져옵니다. 
    - 예를 들어, 프로그램을 종료할 때 사용합니다.
- `typing`: 함수에서 데이터 타입을 명확히 표시하기 위해 사용됩니다.

In [3]:
import subprocess
from typing import List
import re
import os

# ---- 도우미 함수 ----
def run_command(command: str) -> str:
    """
    쉘 명령어를 실행하고 그 결과를 반환합니다.
    """
    try:
        result = subprocess.run(
            command, 
            shell=True, 
            check=False,
            text=True, 
            capture_output=True, 
            encoding='utf-8'
        )
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            print(f"🚫 명령어 실행 중 오류 발생: {command}")
            if result.stderr:
                print(f"🔍 오류 내용: {result.stderr}")
            return ""
    except Exception as e:
        print(f"🚫 명령어 실행 중 예외 발생: {str(e)}")
        return ""

def validate_branch_name(branch_name: str) -> bool:
    """브랜치 이름 검증"""
    pattern = r'^[a-zA-Z0-9-_/]+$'
    return bool(re.match(pattern, branch_name))

def list_existing_branches() -> List[str]:
    """브랜치 목록 조회"""
    branches = run_command("git branch").split("\n")
    return [branch.strip("* ").strip() for branch in branches if branch]

def switch_or_create_branch(branch_name: str) -> bool:
    """브랜치 전환 또는 생성"""
    if not validate_branch_name(branch_name):
        print(f"❌ 유효하지 않은 브랜치 이름입니다: {branch_name}")
        return False

    existing_branches = list_existing_branches()
    
    if branch_name in existing_branches:
        print(f"🔄 기존 브랜치로 전환: {branch_name}")
        result = run_command(f"git checkout {branch_name}")
    else:
        print(f"✨ 새 브랜치 생성: {branch_name}")
        result = run_command(f"git checkout -b {branch_name}")
    
    return bool(result)

def validate_commit_message(message: str) -> bool:
    """커밋 메시지 검증"""
    basic_pattern = r'^\[(feat|fix|chore|docs|style|refactor|test|ci|perf)\].+'
    
    if not re.match(basic_pattern, message):
        return False
    
    if '#' in message:
        issue_key_pattern = r'.*#[A-Z]+-\d+$'
        return bool(re.match(issue_key_pattern, message))
    
    return True

def get_git_root_directory() -> str:
    """Git 저장소의 루트 디렉토리 경로를 반환합니다."""
    try:
        git_root = run_command("git rev-parse --show-toplevel")
        return git_root if git_root else os.getcwd()
    except:
        return os.getcwd()

def get_absolute_path(path: str) -> str:
    """파일 또는 폴더의 절대 경로를 반환합니다."""
    current_dir = os.getcwd()
    git_root = get_git_root_directory()
    
    # 가능한 경로들을 검사
    possible_paths = [
        os.path.join(current_dir, path),  # 현재 디렉토리
        os.path.join(git_root, path),     # Git 루트 디렉토리
        path                              # 입력된 경로 그대로
    ]
    
    # 존재하는 첫 번째 경로 반환
    for possible_path in possible_paths:
        if os.path.exists(possible_path):
            return possible_path
            
    return path  # 경로를 찾지 못한 경우 원래 경로 반환

def commit_paths(paths: List[str], commit_message: str) -> bool:
    """파일과 폴더 커밋"""
    if not validate_commit_message(commit_message):
        print("❌ 유효하지 않은 커밋 메시지 형식입니다.")
        return False

    # 파일/폴더 존재 여부 확인 및 스테이징
    added_paths = []
    for path in paths:
        # 경로의 절대 경로 얻기
        abs_path = get_absolute_path(path)
        
        if os.path.exists(abs_path):
            if os.path.isdir(abs_path):
                # 디렉토리인 경우 모든 내용을 재귀적으로 추가
                result = run_command(f'git add "{abs_path}"')
                if result is not None:
                    added_paths.append(abs_path)
                    print(f"✅ '{path}' 폴더와 그 내용 스테이징 완료")
            else:
                # 파일인 경우
                if '.' not in path and path.strip() != '':
                    # 확장자가 없는 경우 .ipynb를 추가
                    abs_path = abs_path + '.ipynb'
                    path = path + '.ipynb'
                result = run_command(f'git add "{abs_path}"')
                if result is not None:
                    added_paths.append(abs_path)
                    print(f"✅ '{path}' 스테이징 완료")
        else:
            print(f"❌ '{path}' 경로를 찾을 수 없습니다")

    if not added_paths:
        print("⚠️ 커밋할 항목이 없습니다")
        return False
    
    # 커밋 실행
    result = run_command(f'git commit -m "{commit_message}"')
    if result:
        print("✅ 커밋 완료!")
        return True
    return False

def push_branch(branch_name: str) -> bool:
    """브랜치 푸시"""
    print(f"🚀 브랜치 푸시 중: {branch_name}")
    push_result = run_command(f"git push origin {branch_name}")
    verify_result = run_command(f"git ls-remote --heads origin {branch_name}")
    
    if verify_result:
        print(f"✨ 브랜치 {branch_name} 푸시 완료")
        return True
    else:
        print(f"❌ 브랜치 {branch_name} 푸시 실패")
        return False

def main():
    """메인 함수"""
    print(f"\n📂 현재 작업 디렉토리: {os.getcwd()}")
    print(f"📂 Git 루트 디렉토리: {get_git_root_directory()}")

    # 브랜치 옵션
    branch_options = ["main", "dev", "jaeuk", "hyunjung", "seongtae", "seoyun", "hyowon", "jinsil"]

    print("\n🌿 브랜치 선택")
    print("사용 가능한 브랜치:")
    for idx, branch in enumerate(branch_options):
        print(f"{idx + 1}. {branch}")
    
    try:
        branch_choice = int(input("\n번호를 선택하세요 (1-8): ")) - 1
        if branch_choice < 0 or branch_choice >= len(branch_options):
            print("❌ 잘못된 브랜치 번호입니다")
            return
    except ValueError:
        print("❌ 유효한 숫자를 입력하세요")
        return

    selected_branch = branch_options[branch_choice]
    if not switch_or_create_branch(selected_branch):
        return

    # 커밋할 파일/폴더 입력 받기
    print("\n📁 커밋할 파일 또는 폴더")
    paths_input = input("파일/폴더 경로를 쉼표로 구분하여 입력하세요 (예: file1.py, folder1, file2.txt): ")
    paths_to_commit = [p.strip() for p in paths_input.split(",") if p.strip()]

    if not paths_to_commit:
        print("❌ 커밋할 항목이 지정되지 않았습니다")
        return

    # 커밋 메시지 구성
    print("\n✏️ 커밋 메시지 작성")
    commit_type = input("커밋 타입 (feat/fix/chore/docs/style/refactor/test/ci/perf): ").strip()
    commit_summary = input("커밋 요약: ").strip()
    jira_issue_key = input("JIRA 이슈 키 (예: PROJ-123, 선택사항이므로 없으면 Enter): ").strip()

    commit_message = f"[{commit_type}] {commit_summary}"
    if jira_issue_key:
        commit_message += f" #{jira_issue_key}"

    # 커밋 및 푸시 실행
    if commit_paths(paths_to_commit, commit_message):
        push_result = push_branch(selected_branch)
        if push_result:
            print("\n✨ 모든 작업이 완료되었습니다!")
        else:
            print("\n⚠️ 주의: 커밋은 완료되었으나 푸시는 실패했습니다.")
            print(f"💡 수동으로 푸시하려면: git push origin {selected_branch}")
    else:
        print("\n❌ 커밋 중 오류가 발생했습니다")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n⚠️ 사용자에 의해 프로그램이 중단되었습니다")
    except Exception as e:
        print(f"\n❌ 예상치 못한 오류가 발생했습니다: {str(e)}")


📂 현재 작업 디렉토리: c:\Users\SSAFY\Desktop\S12P11B201
📂 Git 루트 디렉토리: C:/Users/SSAFY/Desktop/S12P11B201

🌿 브랜치 선택
사용 가능한 브랜치:
1. main
2. dev
3. jaeuk
4. hyunjung
5. seongtae
6. seoyun
7. hyowon
8. jinsil
🔄 기존 브랜치로 전환: jaeuk

📁 커밋할 파일 또는 폴더

✏️ 커밋 메시지 작성
✅ 'embedded' 폴더와 그 내용 스테이징 완료
🚫 명령어 실행 중 오류 발생: git commit -m "[chore] 임베디드 폴더구조 작성"

❌ 커밋 중 오류가 발생했습니다
